# Process Monitoring dataset

In [20]:
import pandas as pd

# cargar datos en bruto
monitoringdf = pd.read_csv('../data/_Process_monitoring__202303251137.csv', sep=',')

# Creacion del las columnas datetime y latitude_longitude
monitoringdf['datetime'] = monitoringdf['date'] + ' ' + monitoringdf['time']
monitoringdf['latitude_longitude'] = monitoringdf.apply(lambda row: (row['latitude'], row['longitude']), axis=1)

# eliminar las columnas latitude, longitude, time, velocity. Luego reordenar columnas
monitoringdf.drop(columns=['latitude', 'longitude','time'], inplace=True)
monitoringdf = monitoringdf[['id', 'date','datetime', 'latitude_longitude', 'lap', 'navigation', 'vehicle_id']]
monitoringdf

,id,date,datetime,latitude_longitude,lap,navigation,vehicle_id
0,13173398,2023-03-01,2023-03-01 05:10:11,"(-13.5495973, -71.8775099)",1,285,SJ39
1,13177467,2023-03-01,2023-03-01 06:04:51,"(-13.5245292, -71.9775272)",1,298,SJ34
2,13189674,2023-03-01,2023-03-01 07:22:21,"(-13.5341828, -71.9807088)",1,7,SJ29
3,13192729,2023-03-01,2023-03-01 07:39:04,"(-13.5511464, -71.9869284)",1,202,SJ21
4,13193272,2023-03-01,2023-03-01 07:41:52,"(-13.548642, -71.88101)",1,107,SJ34
...,...,...,...,...,...,...,...
1685065,17241845,2023-03-25,2023-03-25 08:50:24,"(-13.5254398, -71.9750786)",2,288,SJ24
1685066,17253753,2023-03-25,2023-03-25 10:01:56,"(-13.5511209, -71.9896213)",2,253,SJ16
1685067,17257644,2023-03-25,2023-03-25 10:26:10,"(-13.5432069, -71.8977847)",2,126,SJ24
1685068,17266468,2023-03-25,2023-03-25 11:14:26,"(-13.5327737, -71.9218652)",1,258,SJ43


### Cleaning

In [21]:
# DUPLICADOS
print(f'Shape dataset: {monitoringdf.shape}')

# Verificar y contar si hay puntos duplicados segun datetime y latitude_longitude
base = ['datetime', 'latitude_longitude']
twocols_dupli = monitoringdf.duplicated(subset=base)
print(f"Nro de duplicados en {base}: {twocols_dupli.sum()}")

# ============================ Solo para comprobacion ============================
# Verificar y contar si hay puntos duplicados segun latitude_longitude
# onecol_dupli =  monitoringdf['latitude_longitude'].duplicated()
# print(len(twocols_dupli), len(onecol_dupli))
# print(twocols_dupli.sum(), onecol_dupli.sum())
# print(abs(twocols_dupli.sum() - onecol_dupli.sum()))
# tmp = [ not x==y for x,y in zip(twocols_dupli, onecol_dupli)]
# data_tmp = monitoringdf.copy()
# data_tmp['dupli'] = tmp
# data_tmp[data_tmp['dupli'] == True]
# ================================================================================

# Eliminar columnas
monitoringdf.drop_duplicates(subset=base, inplace=True)
print(f"Duplicados eliminados => shape dataset: {monitoringdf.shape}")

Shape dataset: (1685070, 7)
Nro de duplicados en ['datetime', 'latitude_longitude']: 13116
Duplicados eliminados => shape dataset: (1671954, 7)


In [22]:
# NANs
# verificar si hay datos faltantes
print("Cantidad de NaNs:")
for col in monitoringdf:
    nans = monitoringdf[col].isna().sum()
    print(f"\tColumna {col}: {nans}")

Cantidad de NaNs:
	Columna id: 0
	Columna date: 0
	Columna datetime: 0
	Columna latitude_longitude: 0
	Columna lap: 0
	Columna navigation: 0
	Columna vehicle_id: 0


# Bus Stops Ida dataset

In [23]:
# leer dataset busstop
busstopdf = pd.read_csv('../data/busstops_ida.csv')

# diseño del df: creacion de latitude_longitude
busstopdf['latitude_longitude'] = busstopdf.apply(lambda row: (row['latitude'], row['longitude']), axis=1)
busstopdf.drop(columns=['number', 'latitude', 'longitude'], inplace=True)

# para fines practicos modificamos el id = index para mejor lectura
busstopdf['id'] = [f'{i+100}' for i in busstopdf.index]

# mostrar info y elementos
print(busstopdf.info())
busstopdf

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 84 entries, 0 to 83
Data columns (total 5 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   id                  84 non-null     object
 1   name                84 non-null     object
 2   navigation          84 non-null     int64 
 3   radio               84 non-null     int64 
 4   latitude_longitude  84 non-null     object
dtypes: int64(2), object(3)
memory usage: 3.4+ KB
None


,id,name,navigation,radio,latitude_longitude
0,100,inicio,159,30,"(-13.5549784, -71.856025)"
1,101,primer stop,304,18,"(-13.5548123, -71.8573001)"
2,102,segundo stop,281,13,"(-13.5532944, -71.8619442)"
3,103,Ccollana,283,9,"(-13.552296, -71.867299)"
4,104,Puente Huaccoto,283,10,"(-13.551592, -71.870211)"
...,...,...,...,...,...
79,179,Segundo,221,10,"(-13.547528, -71.985248)"
80,180,Reservorio,225,10,"(-13.54966, -71.988003)"
81,181,Cuarto,230,10,"(-13.550408, -71.98888)"
82,182,Quinto,203,10,"(-13.550841, -71.989448)"


### Crear links

In [24]:
# creamos dataframe links y renombramos columnas
tmp1 = busstopdf[['id','name']].head(busstopdf.shape[0]-1).reset_index(drop=True)
# tmp1.rename(columns={'id':'ini_id', 'name':'ini_name'}, inplace=True)
tmp1.rename(columns={'id':'init_stop', 'name':'init_name'}, inplace=True)

tmp2 = busstopdf[['id','name']].tail(busstopdf.shape[0]-1).reset_index(drop=True)
# tmp2.rename(columns={'id':'end_id', 'name':'end_name'}, inplace=True)
tmp2.rename(columns={'id':'end_stop', 'name':'end_name'}, inplace=True)
linksdf = pd.concat([tmp1, tmp2], axis=1)

# creamos la columnas link
# linksdf['link'] = linksdf.apply(lambda row: f"{row['ini_id']}:{row['end_id']}", axis=1)
linksdf['link'] = linksdf.apply(lambda row: f"{row['init_stop']}:{row['end_stop']}", axis=1)
linksdf['id_link'] = linksdf.index
linksdf.insert(0, 'id_link', linksdf.pop('id_link'))
linksdf

,id_link,init_stop,init_name,end_stop,end_name,link
0,0,100,inicio,101,primer stop,100:101
1,1,101,primer stop,102,segundo stop,101:102
2,2,102,segundo stop,103,Ccollana,102:103
3,3,103,Ccollana,104,Puente Huaccoto,103:104
4,4,104,Puente Huaccoto,105,Kayra,104:105
...,...,...,...,...,...,...
78,78,178,Primero,179,Segundo,178:179
79,79,179,Segundo,180,Reservorio,179:180
80,80,180,Reservorio,181,Cuarto,180:181
81,81,181,Cuarto,182,Quinto,181:182


# Calcular Travel Time

In [25]:
from haversine import haversine as hvs, Unit
import numpy as np

# FUNCION: filtrar los puntos de ida
def idaPoints(df):
    end1_stop = {'latitude_longitude':(-13.5513636,-71.988304), 'navigation':75, 'radio':50}
    end2_stop = {'latitude_longitude':(-13.5511395,-71.986925), 'navigation':206, 'radio':50}
    for i, row in df.iterrows():
        if (itPassed(row, end1_stop) or itPassed(row, end2_stop)):
            break
    return df.head(i+1)

# FUNCION: ver si un punto paso por un stop
def itPassed(point, stop):
    nav = abs(point['navigation'] - stop['navigation'])
    dis = round(hvs(point['latitude_longitude'], stop['latitude_longitude'], unit=Unit.METERS), 2)
    return (nav<=45 and dis<=stop['radio'])

# FUNCION: VERIFICAR SI EL PUNTO PASO POR UN STOP
def searchStopsPoints(row, stops, r_nav=45, r_dtec=60):
    # verificar la navegacion y el radio de distancia
    stops = stops[abs(stops['navigation'] - row['navigation']) <= r_nav]
    stops['dis'] = stops['latitude_longitude'].apply(lambda stop: round(hvs(row['latitude_longitude'], stop, unit=Unit.METERS), 2))
    stops = stops[stops['dis'] <= r_dtec]
    
    # preparar el output
    row['stop'] = str(stops.loc[stops['dis'].idxmin(), 'id']) if len(stops) != 0 else np.nan
    row['dis'] = float(stops.loc[stops['dis'].idxmin(), 'dis']) if len(stops) != 0 else 0    
    return row

# FUNCION: CALCULA EL TIEMPO EN SEG DE UN PUNTO CON SU ANTERIOR
def time_travel(row, df):
    if {row['init_stop'], row['end_stop']}.issubset(set(df.index)):
        # print('if')
        ti, te =  df.loc[row['init_stop'], 'datetime'], df.loc[row['end_stop'], 'datetime']
        row['date_time_init'] = ti
        row['date_time_end'] = te
        time = abs((ti - te).total_seconds())
        
        #row['time_travel'] =  time if time < 300 else np.nan
        row['time_travel'] =  time
        row['init_idp'], row['end_idp'] = df.loc[row['init_stop'], 'id'], df.loc[row['end_stop'], 'id']
        row['veh'] = df.loc[row['init_stop'], 'vehicle_id']
        row['lap'] = df.loc[row['init_stop'], 'lap']
        row['date'] = df.loc[row['init_stop'], 'date']

    else:
        # print('else')
        row['date_time_init'] = np.nan
        row['date_time_end'] = np.nan
        row['time_travel'] = np.nan
        row['init_idp'] = np.nan
        row['end_idp'] = np.nan
        row['veh'] = np.nan
        row['lap'] = np.nan
        row['date'] = np.nan
        
    return row

### Crear samples y calcular travels time

In [26]:
# ordenar df por fechas
monitoringdf['datetime'] = pd.to_datetime(monitoringdf['datetime'], format='%Y-%m-%d %H:%M:%S')
monitoringdf = monitoringdf.sort_values(by='datetime')
print("Type columna 'datetime' : ", monitoringdf['datetime'].dtype)
monitoringdf.head()

Type columna 'datetime' :  datetime64[ns]


,id,date,datetime,latitude_longitude,lap,navigation,vehicle_id
29756,13172252,2023-03-01,2023-03-01 00:01:34,"(-13.5513563, -71.9902231)",5,140,SJ40
29311,13172234,2023-03-01,2023-03-01 00:01:49,"(-13.5506013, -71.8786173)",0,185,SJ10
29780,13172253,2023-03-01,2023-03-01 00:03:25,"(-13.5506231, -71.8786667)",0,206,SJ10
31234,13172309,2023-03-01,2023-03-01 00:05:13,"(-13.5383472, -71.9807985)",5,0,SJ08
29831,13172255,2023-03-01,2023-03-01 00:05:30,"(-13.55065, -71.8785454)",0,113,SJ10


In [27]:
# preparar el dataset para iterar en todos los posibles samples
monitoringdf_tmp = monitoringdf.copy()
#monitoringdf_tmp = monitoringdf_tmp[(monitoringdf_tmp['date'] == '2023-03-01')] #| (monitoringdf_tmp['date'] == '2023-03-02')]
monitoringdf_tmp['sample'] = monitoringdf.apply(lambda row: f"{row['date']} {row['vehicle_id']} {row['lap']}" , axis=1)
n_posibles_samples = len(monitoringdf_tmp['sample'].unique())
print('nro posibles samples:', n_posibles_samples)

# iterar en todos los posibles
n_descartados = 0
id_sample = 0
list_err = []
list_samplesdf = []
date = ''
for ids, pointsdf in monitoringdf_tmp.groupby('sample'):
    # Seguir procesamiento
    if date != pointsdf['date'].unique()[0]:
        date = pointsdf['date'].unique()[0]
        print(date)

    # ordenar el sample por datetime
    pointsdf.sort_values(by='datetime', inplace=True)

    # identificar los puntos de ida del sample
    idaPointsdf = idaPoints(pointsdf.reset_index(drop=True))

    # verificar puntos por stop
    stopsPointsdf = idaPointsdf.apply(lambda row: searchStopsPoints(row, busstopdf), axis=1)
    stopsPointsdf = stopsPointsdf[stopsPointsdf['stop'].notna()].reset_index()

    # Seleccionar un unico punto por stops
    stopsPointsdf.sort_values(by=['stop', 'dis'], ascending=True, inplace=True, ignore_index=True, key=np.int64)
    stopsPointsdf.drop_duplicates(subset=['stop'], keep='first', inplace=True, ignore_index=True)

    # filtrar los stop sin puntos 
    missing_stops = list(set(busstopdf['id']) - set(stopsPointsdf['stop'].unique()))

    # verificar la cantidad de puntos filtrados
    if len(missing_stops) < 24:
        # verificar la secuencialidad de los datetime
        byS = list(stopsPointsdf.sort_values(by='stop')['stop'])
        byT = list(stopsPointsdf.sort_values(by='datetime')['stop'])
        if byS == byT:
            # crear sample y calcular el travel_time
            sampledf = linksdf.copy()
            stopsPointsdf.set_index('stop', inplace=True)
            sampledf = sampledf.apply(lambda row: time_travel(row, stopsPointsdf), axis=1)
            
            # crear el id_sample
            # sampledf['id_sample'] = 
            sampledf.insert(0, 'id_sample', [id_sample] * sampledf.shape[0])
            list_samplesdf.append(sampledf)
            id_sample += 1
        else:
            # registrar sample defectoso
            list_err.append(stopsPointsdf)
            print(' ', ids)
    else:
        n_descartados += 1

# mostrar nro de samples encontrados
print('='*100)
print(f"{len(list_samplesdf)} Samples buenos")
print(f"{len(list_err)} Samples con errores de continuidad de tiempo")
print(f"{n_descartados} Samples descartados por falta de puntos")

nro posibles samples: 5015
2023-03-01
  2023-03-01 SJ13 4
  2023-03-01 SJ29 1
  2023-03-01 SJ40 5
2023-03-02
  2023-03-02 SJ08 3
  2023-03-02 SJ09 1
  2023-03-02 SJ23 3
  2023-03-02 SJ23 4
  2023-03-02 SJ32 4
  2023-03-02 SJ36 2
  2023-03-02 SJ36 5
2023-03-03
  2023-03-03 SJ23 3
  2023-03-03 SJ29 5
  2023-03-03 SJ40 2
2023-03-04
  2023-03-04 SJ12 6
2023-03-05
2023-03-06
  2023-03-06 SJ13 5
  2023-03-06 SJ23 2
  2023-03-06 SJ35 2
2023-03-07
  2023-03-07 SJ08 2
  2023-03-07 SJ18 5
2023-03-08
2023-03-09
2023-03-10
  2023-03-10 SJ29 4
  2023-03-10 SJ36 5
2023-03-11
2023-03-12
  2023-03-12 SJ20 4
  2023-03-12 SJ29 3
  2023-03-12 SJ31 1
  2023-03-12 SJ31 3
  2023-03-12 SJ34 5
  2023-03-12 SJ36 5
  2023-03-12 SJ43 4
2023-03-13
  2023-03-13 SJ03 5
  2023-03-13 SJ11 6
  2023-03-13 SJ13 4
  2023-03-13 SJ25 3
2023-03-14
2023-03-15
2023-03-16
2023-03-17
2023-03-18
  2023-03-18 SJ04 3
  2023-03-18 SJ08 3
2023-03-19
  2023-03-19 SJ06 2
  2023-03-19 SJ27 3
  2023-03-19 SJ34 3
  2023-03-19 SJ38 3
2023

In [28]:
# Reparar samples con error de secuencialidad de tiempo
# (los samples reparados se debe agregar a 'list_samplesdf')

In [31]:
samples_nans = pd.concat(list_samplesdf)
samples_nans

,id_sample,id_link,init_stop,init_name,end_stop,end_name,link,date_time_init,date_time_end,time_travel,init_idp,end_idp,veh,lap,date
0,0,0,100,inicio,101,primer stop,100:101,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN
1,0,1,101,primer stop,102,segundo stop,101:102,2023-03-01 07:27:33,2023-03-01 07:29:19,106.0,13190622.0,13190922.0,SJ01,1.0,2023-03-01
2,0,2,102,segundo stop,103,Ccollana,102:103,2023-03-01 07:29:19,2023-03-01 07:30:49,90.0,13190922.0,13191197.0,SJ01,1.0,2023-03-01
3,0,3,103,Ccollana,104,Puente Huaccoto,103:104,2023-03-01 07:30:49,2023-03-01 07:31:13,24.0,13191197.0,13191276.0,SJ01,1.0,2023-03-01
4,0,4,104,Puente Huaccoto,105,Kayra,104:105,2023-03-01 07:31:13,2023-03-01 07:32:29,76.0,13191276.0,13191535.0,SJ01,1.0,2023-03-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
78,3360,78,178,Primero,179,Segundo,178:179,2023-03-25 11:08:40,2023-03-25 11:08:52,12.0,17265203.0,17265239.0,SJ42,2.0,2023-03-25
79,3360,79,179,Segundo,180,Reservorio,179:180,2023-03-25 11:08:52,2023-03-25 11:09:16,24.0,17265239.0,17265310.0,SJ42,2.0,2023-03-25
80,3360,80,180,Reservorio,181,Cuarto,180:181,2023-03-25 11:09:16,2023-03-25 11:09:28,12.0,17265310.0,17265340.0,SJ42,2.0,2023-03-25
81,3360,81,181,Cuarto,182,Quinto,181:182,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN


In [32]:
# Convertir a csv
pd.concat(list_samplesdf).to_csv('../data/samples_nans.csv', index=False)